In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [2]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
submission = pd.read_csv('sample_submission.csv')

In [3]:
train= train.drop('id', axis=1)
test = test.drop('id',axis=1)

In [4]:
from sklearn.preprocessing import LabelEncoder

In [5]:
encoder= LabelEncoder().fit(train['target'])

In [6]:
train['target'] = encoder.transform(train['target'])

In [7]:
x = train.drop(['target'],axis=1)

In [8]:
y = train['target']

In [9]:
x.head(3)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,0,0,1,0,1,0,0,0,0,0,...,3,0,0,21,0,0,0,0,0,0
1,0,0,0,0,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,2,...,0,0,1,0,0,0,0,13,2,0


In [10]:
y.head(3)

0    1
1    0
2    0
Name: target, dtype: int64

In [11]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [12]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 3, 5, 7],
        'gamma': [0, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.3, 0.5, 0.7, 0.9, 1],
        'subsample': [0.6, 0.7, 0.8, 0.9, 1],
        'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1],
        'max_depth': [3, 5, 6, 7, 9, 12, 15, 17, 25],
    'learning_rate': [0.01, 0.015, 0.025, 0.05, 0.1]
        }

In [13]:
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [14]:
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1997)

In [15]:
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(80000, 50) (80000,) (20000, 50) (20000,)


In [16]:
model = XGBClassifier(
                      num_class=4, 
                      objective='multi:softprob',
                      tree_method='gpu_hist',
                      n_estimators=600,
                      use_label_encoder =False,
                      eval_metric='mlogloss')
                      #predictor='gpu_predictor')

In [17]:
print(model)

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              eval_metric='mlogloss', gamma=None, gpu_id=None,
              importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=600, n_jobs=None, num_class=4,
              num_parallel_tree=None, objective='multi:softprob',
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method='gpu_hist',
              use_label_encoder=False, validate_parameters=None,
              verbosity=None)


In [18]:
param_comb=5
folds=3
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1997)
random_search = RandomizedSearchCV(model, param_distributions=params, n_iter=param_comb, n_jobs=16, cv=skf.split(x,y), verbose=3, random_state=1001 )
# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(x, y)
timer(start_time) # timing ends here for "start_time" variable
#print('\n All results:')
#print(random_search.cv_results_)
#print('\n Randomsearch Best estimator:')
#print(random_search.best_estimator_)
#print('\n Randomsearch Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
#print(random_search.best_score_ * 2 - 1)
print('\n Randomsearch Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('random-grid-search-{}folds.csv'.format(folds), index=False)

random_search = random_search.best_estimator_
test_y = random_search.predict_proba(test)
index = submission.columns.values.tolist()
random_submission = pd.DataFrame(test_y,columns = index[1:5])
random_submission['id'] = submission['id']
submission = random_submission[['id','Class_1','Class_2','Class_3','Class_4']]
submission.to_csv('submission{}.csv'.format(folds),index=False)


Fitting 3 folds for each of 5 candidates, totalling 15 fits

 Time taken: 0 hours 6 minutes and 51.53 seconds.

 Randomsearch Best hyperparameters:
{'subsample': 0.6, 'min_child_weight': 7, 'max_depth': 6, 'learning_rate': 0.05, 'gamma': 0, 'colsample_bytree': 0.9}


In [19]:
param_comb=5
folds=4
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1997)
random_search = RandomizedSearchCV(model, param_distributions=params, n_iter=param_comb, n_jobs=16, cv=skf.split(x,y), verbose=3, random_state=1001 )
# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(x, y)
timer(start_time) # timing ends here for "start_time" variable
#print('\n All results:')
#print(random_search.cv_results_)
#print('\n Randomsearch Best estimator:')
#print(random_search.best_estimator_)
#print('\n Randomsearch Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
#print(random_search.best_score_ * 2 - 1)
print('\n Randomsearch Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('random-grid-search-{}folds.csv'.format(folds), index=False)

random_search = random_search.best_estimator_
test_y = random_search.predict_proba(test)
index = submission.columns.values.tolist()
random_submission = pd.DataFrame(test_y,columns = index[1:5])
random_submission['id'] = submission['id']
submission = random_submission[['id','Class_1','Class_2','Class_3','Class_4']]
submission.to_csv('submission{}.csv'.format(folds),index=False)

Fitting 4 folds for each of 5 candidates, totalling 20 fits

 Time taken: 0 hours 9 minutes and 5.13 seconds.

 Randomsearch Best hyperparameters:
{'subsample': 0.6, 'min_child_weight': 7, 'max_depth': 6, 'learning_rate': 0.05, 'gamma': 0, 'colsample_bytree': 0.9}


In [20]:
param_comb=5
folds=5
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1997)
random_search = RandomizedSearchCV(model, param_distributions=params, n_iter=param_comb, n_jobs=16, cv=skf.split(x,y), verbose=3, random_state=1001 )
# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(x, y)
timer(start_time) # timing ends here for "start_time" variable
#print('\n All results:')
#print(random_search.cv_results_)
#print('\n Randomsearch Best estimator:')
#print(random_search.best_estimator_)
#print('\n Randomsearch Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
#print(random_search.best_score_ * 2 - 1)
print('\n Randomsearch Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('random-grid-search-{}folds.csv'.format(folds), index=False)

random_search = random_search.best_estimator_
test_y = random_search.predict_proba(test)
index = submission.columns.values.tolist()
random_submission = pd.DataFrame(test_y,columns = index[1:5])
random_submission['id'] = submission['id']
submission = random_submission[['id','Class_1','Class_2','Class_3','Class_4']]
submission.to_csv('submission{}.csv'.format(folds),index=False)

Fitting 5 folds for each of 5 candidates, totalling 25 fits

 Time taken: 0 hours 11 minutes and 47.11 seconds.

 Randomsearch Best hyperparameters:
{'subsample': 0.6, 'min_child_weight': 7, 'max_depth': 6, 'learning_rate': 0.05, 'gamma': 0, 'colsample_bytree': 0.9}


In [22]:

folds=5
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1997) 
grid = GridSearchCV(estimator=model, param_grid=params, n_jobs=16, cv=skf.split(x,y), verbose=3 )
grid.fit(x, y)
#print('\n All results:')
#print(grid.cv_results_)
#print('\n Grid Best estimator:')
#print(grid.best_estimator_)
#print('\n Grid Best score:')
#print(grid.best_score_ * 2 - 1)
print('\n Grid Best parameters:')
print(grid.best_params_)
gresults = pd.DataFrame(grid.cv_results_)
gresults.to_csv('Grid-search-folds{}.csv'.format(folds), index=False)

grid = grid.best_estimator_
test_y_grid = grid.predict_proba(test)
index = submission.columns.values.tolist()
grid_submission = pd.DataFrame(test_y_grid,columns = index[1:5])
grid_submission['id'] = submission['id']
submission2 = grid_submission[['id','Class_1','Class_2','Class_3','Class_4']]
submission2.to_csv('Gsubmission{}.csv'.foramt(folds),index=False)

IndentationError: unexpected indent (<ipython-input-22-6d1f23e4e361>, line 2)